# モデルの精度を確認するためのインターフェース

ver.2022/10/30

学習したモデルでTTSと非リアルタイムのVCを行い、モデルの精度を検証します。

​

In [ ]:
#@title ## 1 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
print("log: パッケージのインストールが完了しました。\n")

​

In [ ]:
#@title ## 2 リポジトリの準備
#@markdown リポジトリの準備を行います。

#@markdown まず、セルを実行してください。UIが表示されます。

#@markdown Colabを使用している場合
#@markdown * PlatformでColabを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、Google Driveのマイドライブ以下のパスを指定してください。よく分からない場合は、変更しなくとも構いません。変更しない場合は、マイドライブ直下に保存されます。
#@markdown * 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown ローカルの場合
#@markdown * PlatformでLocalを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、ローカルのパスを指定してください。

#@markdown 設定が完了したら、次へを押してください。


#---関数---
def mount_googledrive():
  print("log: Google Driveのマウントを開始します。")
  print("「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。")
  time.sleep(2)
  print("info: 少し時間がかかります。このままお待ちください。")
  from google.colab import drive
  drive.mount('/content/drive')
  print("log: Google Driveのマウントが完了しました。\n")

#---関数終わり---

def first_view():
  #widgetsの構成
  global platform_input #グローバル指定
  #global directory_input #グローバル指定
  #global branch_input #グローバル指定
  global path_input #グローバル指定
  platform_input = widgets.Dropdown(options=["Colab", "Local"], value="Colab", description='Platform:', disabled=False)
  path_input = widgets.Text(value='/content/drive/MyDrive/MMVC_Trainer_main', placeholder='/content/drive/MyDrive/MMVC_Trainer_main', description='Path:', disabled=False)
  #directory_input = widgets.Text(value='/content/drive/MyDrive/MMVC_Trainer', placeholder='/content/drive/MyDrive/MMVC_Trainer', description='Directory:', disabled=False)
  #branch_input = widgets.Text(value='main', placeholder='main', description='Branch:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(platform_input, path_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  #global directory #グローバル指定
  #directory = directory_input.value
  global platform #グローバル指定
  platform = platform_input.value
  #global branch #グローバル指定
  #branch = branch_input.value
  global path #グローバル指定
  #path = directory + "/MMVC_Trainer_" + branch
  path = path_input.value
  #Colabとそれ以外で処理を分ける
  if platform == "Colab":
    mount_googledrive() #Google Driveのマウント
    %cd $path
  elif platform == "Local":
    %cd $path

#実行
first_view()

​

In [ ]:
#@title ## 3 ライブラリのインストール
#@markdown 実行に必要なライブラリを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

!apt-get install espeak
!pip install -r requirements.txt

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

​

In [ ]:
#@title ## 4 学習したモデルの読み込み
#@markdown 学習したモデルを読み込みます。

#@markdown まず、セルを実行してください。

#@markdown UIが表示されますので、以下の説明に従って設定を行ってください。

#@markdown ​

#@markdown CONFIG_PATH：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。

#@markdown NET_PATH：学習したモデルのパス
#@markdown 作成されたモデル(pth)を指定してください。
#@markdown `logs/xxxxxxxx_xxxxx/G_xxxxx.pth` のような値になります。

#@markdown ​

#@markdown 設定が完了したら、次へを押してください。

def first_view():
  #widgetsの構成
  global CONFIG_PATH_input #グローバル指定
  global NET_PATH_input #グローバル指定
  CONFIG_PATH_input = widgets.Text(value='configs/train_config.json', placeholder='configs/train_config.json', description='config_path:', disabled=False)
  NET_PATH_input = widgets.Text(value='logs/20220306_24000/G_xxxxx.pth', placeholder='logs/20220306_24000/G_xxxxx.pth', description='net_path:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(CONFIG_PATH_input, NET_PATH_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global CONFIG_PATH #グローバル指定
  CONFIG_PATH = CONFIG_PATH_input.value
  global NET_PATH #グローバル指定
  NET_PATH = NET_PATH_input.value
  #モデルの読み込み
  global hps
  hps = utils.get_hparams_from_file(CONFIG_PATH)
  global net_g
  net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
  #global
  _ = net_g.eval()
  #global
  _ = utils.load_checkpoint(NET_PATH, net_g, None)

  

#実行
first_view()

​

In [ ]:
#@title ## 5 学習したモデルで非リアルタイムVCを行う
#@markdown 非リアルタイムのVCを行います。

#@markdown まず、セルを実行してください。

#@markdown UIが表示されますので、以下の説明に従って設定を行ってください。

#@markdown ​

#@markdown SOURCE_SPEAKER_ID
#@markdown ソース話者のID
SOURCE_SPEAKER_ID = 107 #@param {type:"integer"}

#@markdown SOURCE_WAVFILE
#@markdown ソース話者の音声ファイルのパス
#@markdown 学習に使用した音声ファイルか、新規に録音した音声ファイルを指定してください。
SOURCE_WAVFILE = "dataset/textful/00_myvoice/wav/VOICEACTRESS100_001.wav" #@param {type:"string"}

#@markdown TARGET_ID
#@markdown ターゲット話者のID
TARGET_ID = 100 #@param {type:"integer"}

#@markdown ​

#@markdown 設定が完了したら、次へを押してください。

#@markdown VCの性能が悪い場合、学習不足か他に問題があります。

with torch.no_grad():
    dataset = TextAudioSpeakerLoader(hps.data.validation_files_notext, hps.data)
    data = dataset.get_audio_text_speaker_pair([SOURCE_WAVFILE, SOURCE_SPEAKER_ID, "a"])
    data = TextAudioSpeakerCollate()([data])
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x for x in data]
    sid_tgt1 = torch.LongTensor([TARGET_ID])
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate))